# Tarea 6: Regresión Lineal Múltiple con Step Forward Feature Selection

**Asignatura:** Métodos Numéricos  
**Fecha:** Noviembre 2025  
**Dataset:** Retail Insights - A Comprehensive Sales Dataset

---

## Resumen Ejecutivo

Este informe presenta un análisis de regresión lineal múltiple aplicado a datos de ventas retail con el objetivo de predecir el total de ventas por pedido. Se implementa el algoritmo **Step Forward Feature Selection (SFFS)** para identificar de manera iterativa las variables más relevantes que expliquen las ventas totales. Los resultados muestran el conjunto óptimo de variables predictoras y las métricas de desempeño del modelo final.

---

## 1. Introducción

### 1.1 Contexto del Problema

En el contexto del comercio minorista, la capacidad de predecir con precisión las ventas es fundamental para la planificación estratégica, gestión de inventarios y toma de decisiones financieras. Este estudio analiza un dataset exhaustivo de transacciones de ventas retail con el objetivo de construir un modelo predictivo robusto.

### 1.2 Objetivos

Los objetivos principales de este análisis son:

1. **Explorar y comprender** la estructura del dataset de ventas retail
2. **Implementar el algoritmo SFFS** para selección automática de variables predictoras
3. **Construir un modelo de regresión lineal múltiple** que prediga el total de ventas por pedido
4. **Evaluar el desempeño** del modelo mediante métricas estándar (MAE, MSE, R²)
5. **Identificar las variables más influyentes** en la predicción de ventas

### 1.3 Metodología

Se utilizará el algoritmo **Step Forward Feature Selection (SFFS)**, que agrega variables de forma iterativa, seleccionando en cada paso aquella que produzca la mayor mejora en el error absoluto medio (MAE). Este enfoque greedy permite identificar las variables más relevantes de manera sistemática.

---

## 2. Desarrollo

### 2.1 Importación de Librerías

In [5]:
# Librerías para manipulación de datos
import pandas as pd
import numpy as np

# Librerías para visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Librerías para modelado
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import LabelEncoder

# Configuración de visualización
import warnings
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

### 2.2 Carga y Exploración Inicial del Dataset

In [6]:
df = pd.read_csv('data.csv')

print("="*80)
print("EXPLORACIÓN INICIAL DEL DATASET")
print("="*80)
print(f"\nDimensiones del dataset: {df.shape[0]} filas x {df.shape[1]} columnas")
print(f"\nPrimeras 5 filas del dataset:")
print("-"*80)
display(df.head())

print(f"\nInformación general del dataset:")
print("-"*80)
df.info()

EXPLORACIÓN INICIAL DEL DATASET

Dimensiones del dataset: 5000 filas x 24 columnas

Primeras 5 filas del dataset:
--------------------------------------------------------------------------------


,Order No,Order Date,Customer Name,Address,City,State,Customer Type,Account Manager,Order Priority,Product Name,...,Cost Price,Retail Price,Profit Margin,Order Quantity,Sub Total,Discount %,Discount $,Order Total,Shipping Cost,Total
0,4293-1,02-09-2014,Vivek Sundaresam,"152 Bunnerong Road,Eastgardens",Sydney,NSW,Small Business,Tina Carlton,Critical,UGen Ultra Professional Cordless Optical Suite,...,$156.50,$300.97,$144.47,23.0,"$4,533.52",2%,$194.83,"$4,757.22",$7.18,"$4,291.55"
1,5001-1,24-10-2015,Shahid Hopkins,"438 Victoria Avenue,Chatswood",Sydney,NSW,Corporate,Natasha Song,Medium,Bagged Rubber Bands,...,$0.24,$1.26,$1.02,8.0,$45.20,3%,$0.00,$45.90,$0.70,$46.91
2,5004-1,13-03-2014,Dennis Pardue,"412 Brunswick St,Fitzroy",Melbourne,VIC,Consumer,Connor Betts,Not Specified,TechSavi Cordless Navigator Duo,...,$42.11,$80.98,$38.87,45.0,$873.32,4%,$72.23,$837.57,$7.18,$82.58
3,5009-1,18-02-2013,Sean Wendt,"145 Ramsay St,Haberfield",Sydney,NSW,Small Business,Phoebe Gour,Critical,Artisan Printable Repositionable Plastic Tabs,...,$5.33,$8.60,$3.27,16.0,$73.52,1%,$4.35,$740.67,$6.19,$730.92
4,5010-1,13-09-2014,Christina Vanderzanden,"188 Pitt Street,Sydney",Sydney,NSW,Small Business,Tina Carlton,Not Specified,Pizazz Drawing Pencil Set,...,$1.53,$2.78,$1.25,49.0,$138.46,7%,$5.95,$123.77,$1.34,$125.97



Información general del dataset:
--------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Order No           5000 non-null   object 
 1   Order Date         5000 non-null   object 
 2   Customer Name      5000 non-null   object 
 3   Address            4999 non-null   object 
 4   City               5000 non-null   object 
 5   State              5000 non-null   object 
 6   Customer Type      5000 non-null   object 
 7   Account Manager    5000 non-null   object 
 8   Order Priority     5000 non-null   object 
 9   Product Name       5000 non-null   object 
 10  Product Category   5000 non-null   object 
 11  Product Container  5000 non-null   object 
 12  Ship Mode          5000 non-null   object 
 13  Ship Date          5000 non-null   object 
 14  Cost 

In [7]:
# Estadísticas descriptivas
print("\nEstadísticas descriptivas de variables numéricas:")
print("-"*80)
display(df.describe())

# Verificar valores nulos
print("\nValores nulos por columna:")
print("-"*80)
nulls = df.isnull().sum()
if nulls.sum() > 0:
    print(nulls[nulls > 0])
else:
    print("No se encontraron valores nulos")


Estadísticas descriptivas de variables numéricas:
--------------------------------------------------------------------------------


,Order Quantity
count,4999.000000
mean,26.483097
std,14.391863
min,1.000000
25%,13.000000
50%,27.000000
75%,39.000000
max,50.000000



Valores nulos por columna:
--------------------------------------------------------------------------------
Address           1
Order Quantity    1
dtype: int64


### 2.3 Preparación de Datos y Feature Engineering

In [8]:
print("="*80)
print("PREPARACIÓN DE DATOS")
print("="*80)

# Crear una copia del dataframe para trabajar
df_work = df.copy()

# Identificar la variable objetivo
print("\nIdentificando variable objetivo...")
print(f"Columnas disponibles: {list(df_work.columns)}")

# Identificar variables numéricas y categóricas
numeric_cols = df_work.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df_work.select_dtypes(include=['object']).columns.tolist()

print(f"\nVariables numéricas ({len(numeric_cols)}): {numeric_cols}")
print(f"Variables categóricas ({len(categorical_cols)}): {categorical_cols}")

# Codificar variables categóricas usando LabelEncoder
print("\nCodificando variables categóricas...")
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df_work[col + '_encoded'] = le.fit_transform(df_work[col].astype(str))
    label_encoders[col] = le
    print(f"  - {col} -> {col}_encoded")

print("\nPreparación de datos completada")

PREPARACIÓN DE DATOS

Identificando variable objetivo...
Columnas disponibles: ['Order No', 'Order Date', 'Customer Name', 'Address', 'City', 'State', 'Customer Type', 'Account Manager', 'Order Priority', 'Product Name', 'Product Category', 'Product Container', 'Ship Mode', 'Ship Date', 'Cost Price', 'Retail Price', 'Profit Margin', 'Order Quantity', 'Sub Total', 'Discount %', 'Discount $', 'Order Total', 'Shipping Cost', 'Total']

Variables numéricas (1): ['Order Quantity']
Variables categóricas (23): ['Order No', 'Order Date', 'Customer Name', 'Address', 'City', 'State', 'Customer Type', 'Account Manager', 'Order Priority', 'Product Name', 'Product Category', 'Product Container', 'Ship Mode', 'Ship Date', 'Cost Price', 'Retail Price', 'Profit Margin', 'Sub Total', 'Discount %', 'Discount $', 'Order Total', 'Shipping Cost', 'Total']

Codificando variables categóricas...
  - Order No -> Order No_encoded
  - Order Date -> Order Date_encoded
  - Customer Name -> Customer Name_encoded
  -

### 2.4 Definición de Variables para el Modelo

Definimos la variable objetivo (Y) y las variables predictoras potenciales (X).

In [9]:
print("="*80)
print("DEFINICIÓN DE VARIABLES")
print("="*80)

# Definir la variable objetivo (Y)
# Asumiendo que la variable objetivo se llama 'Total Amount' o similar
# Ajustar según el nombre real en el dataset

target_column = 'Total Amount'  # Ajustar según el dataset real
if target_column not in df_work.columns:
    # Buscar columnas que puedan ser el total
    possible_targets = [col for col in df_work.columns if 'total' in col.lower() or 'amount' in col.lower()]
    if possible_targets:
        target_column = possible_targets[0]
        print(f"NOTA: Columna objetivo ajustada a: '{target_column}'")

y = df_work[target_column]

# Definir las variables predictoras (X)
# Excluir la variable objetivo y columnas no numéricas originales
exclude_cols = [target_column] + categorical_cols + ['Transaction ID', 'Date'] if 'Transaction ID' in df_work.columns and 'Date' in df_work.columns else [target_column] + categorical_cols

# Obtener todas las columnas numéricas excepto la variable objetivo
potential_features = [col for col in df_work.columns if col not in exclude_cols and (df_work[col].dtype in [np.number, 'int64', 'float64'])]

print(f"\nVariable Objetivo (Y): {target_column}")
print(f"Variables Predictoras Potenciales ({len(potential_features)}):")
for i, feat in enumerate(potential_features, 1):
    print(f"   {i}. {feat}")

# Crear el dataframe X con las variables predictoras
X_all = df_work[potential_features].copy()

print(f"\nDataset preparado: {X_all.shape[0]} observaciones, {X_all.shape[1]} variables predictoras")

DEFINICIÓN DE VARIABLES
NOTA: Columna objetivo ajustada a: 'Sub Total'

Variable Objetivo (Y): Sub Total
Variables Predictoras Potenciales (24):
   1. Order Quantity
   2. Order No_encoded
   3. Order Date_encoded
   4. Customer Name_encoded
   5. Address_encoded
   6. City_encoded
   7. State_encoded
   8. Customer Type_encoded
   9. Account Manager_encoded
   10. Order Priority_encoded
   11. Product Name_encoded
   12. Product Category_encoded
   13. Product Container_encoded
   14. Ship Mode_encoded
   15. Ship Date_encoded
   16. Cost Price_encoded
   17. Retail Price_encoded
   18. Profit Margin_encoded
   19. Sub Total_encoded
   20. Discount %_encoded
   21. Discount $_encoded
   22. Order Total_encoded
   23. Shipping Cost_encoded
   24. Total_encoded

Dataset preparado: 5000 observaciones, 24 variables predictoras


### 2.5 Implementación del Algoritmo Step Forward Feature Selection (SFFS)

El algoritmo SFFS funciona de la siguiente manera:

1. **Inicialización**: Se comienza con un conjunto vacío de variables seleccionadas
2. **Iteración**: En cada paso, se evalúan todas las variables restantes
3. **Selección**: Se elige la variable que, al agregarla al conjunto actual, produce el menor MAE
4. **Actualización**: Se agrega la variable seleccionada y se repite hasta agotar todas las variables
5. **Resultado**: Se obtiene una secuencia ordenada de variables por importancia

In [10]:
print("="*80)
print("STEP FORWARD FEATURE SELECTION (SFFS)")
print("="*80)

import time

# Listas para almacenar resultados
selected_vars = []  # Variables seleccionadas en orden
remaining_vars = potential_features.copy()  # Variables restantes por evaluar
mae_history = []  # Historial de MAE en cada iteración
iteration_details = []  # Detalles de cada iteración

print(f"\nIniciando SFFS con {len(remaining_vars)} variables candidatas...")
print(f"Esto puede tomar varios minutos dependiendo del tamaño del dataset...\n")

start_time = time.time()

# Algoritmo SFFS
iteration = 0
while remaining_vars:
    iteration += 1
    best_mae = float('inf')
    best_var = None
    
    print(f"Iteración {iteration}/{len(potential_features)}")
    print(f"   Variables seleccionadas hasta ahora: {len(selected_vars)}")
    print(f"   Variables restantes por evaluar: {len(remaining_vars)}")
    
    # Probar cada variable restante
    for var in remaining_vars:
        # Crear conjunto temporal de variables (seleccionadas + la variable candidata)
        temp_vars = selected_vars + [var]
        X_temp = X_all[temp_vars]
        
        # Entrenar modelo
        lr = LinearRegression()
        lr.fit(X_temp, y)
        
        # Predecir y calcular MAE
        y_pred = lr.predict(X_temp)
        mae = mean_absolute_error(y, y_pred)
        
        # Actualizar mejor variable si es mejor que la actual
        if mae < best_mae:
            best_mae = mae
            best_var = var
    
    # Agregar la mejor variable encontrada
    selected_vars.append(best_var)
    remaining_vars.remove(best_var)
    mae_history.append(best_mae)
    
    iteration_details.append({
        'Iteración': iteration,
        'Variable Agregada': best_var,
        'MAE': best_mae,
        'Variables Acumuladas': len(selected_vars)
    })
    
    print(f"   Mejor variable: '{best_var}' con MAE = {best_mae:,.2f}\n")

end_time = time.time()
elapsed_time = end_time - start_time

print("="*80)
print(f"SFFS COMPLETADO en {elapsed_time:.2f} segundos ({elapsed_time/60:.2f} minutos)")
print("="*80)

STEP FORWARD FEATURE SELECTION (SFFS)

Iniciando SFFS con 24 variables candidatas...
Esto puede tomar varios minutos dependiendo del tamaño del dataset...

Iteración 1/24
   Variables seleccionadas hasta ahora: 0
   Variables restantes por evaluar: 24


ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

### 2.6 Análisis de Resultados del SFFS

In [11]:
# Crear DataFrame con los detalles de cada iteración
results_df = pd.DataFrame(iteration_details)

print("\nTABLA DE RESULTADOS POR ITERACIÓN")
print("="*80)
display(results_df)

# Identificar el mejor conjunto de variables (menor MAE)
best_idx = np.argmin(mae_history)
best_mae = mae_history[best_idx]
best_n_vars = best_idx + 1
best_vars = selected_vars[:best_n_vars]

print(f"\nMEJOR MODELO ENCONTRADO:")
print("="*80)
print(f"Número de variables: {best_n_vars}")
print(f"MAE mínimo: {best_mae:,.2f}")
print(f"\nVariables del mejor modelo (en orden de importancia):")
for i, var in enumerate(best_vars, 1):
    print(f"   {i}. {var}")
print("="*80)


TABLA DE RESULTADOS POR ITERACIÓN


""


ValueError: attempt to get argmin of an empty sequence

### 2.7 Visualización de la Evolución del MAE

In [ ]:
# Gráfico de evolución del MAE
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.plot(range(1, len(mae_history) + 1), mae_history, marker='o', linewidth=2, markersize=6, color='steelblue')
plt.axvline(x=best_n_vars, color='red', linestyle='--', linewidth=2, label=f'Mejor modelo ({best_n_vars} vars)')
plt.axhline(y=best_mae, color='green', linestyle='--', linewidth=1, alpha=0.5, label=f'MAE mínimo = {best_mae:,.2f}')
plt.xlabel('Número de Variables', fontsize=12)
plt.ylabel('MAE (Mean Absolute Error)', fontsize=12)
plt.title('Evolución del MAE durante SFFS', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend()

plt.subplot(1, 2, 2)
# Calcular la mejora porcentual del MAE
mae_improvement = [(mae_history[0] - mae) / mae_history[0] * 100 for mae in mae_history]
plt.plot(range(1, len(mae_improvement) + 1), mae_improvement, marker='s', linewidth=2, markersize=6, color='forestgreen')
plt.axvline(x=best_n_vars, color='red', linestyle='--', linewidth=2, label=f'Mejor modelo ({best_n_vars} vars)')
plt.xlabel('Número de Variables', fontsize=12)
plt.ylabel('Mejora del MAE (%)', fontsize=12)
plt.title('Mejora Porcentual del MAE', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend()

plt.tight_layout()
plt.show()

print(f"\nMejora total del MAE: {mae_improvement[-1]:.2f}% (desde {mae_history[0]:,.2f} hasta {mae_history[-1]:,.2f})")

### 2.8 Evaluación del Modelo Óptimo

Evaluamos el modelo con el conjunto óptimo de variables identificado por SFFS.

In [ ]:
print("="*80)
print("EVALUACIÓN DEL MODELO ÓPTIMO")
print("="*80)

# Entrenar el modelo final con las mejores variables
X_optimal = X_all[best_vars]
lr_final = LinearRegression()
lr_final.fit(X_optimal, y)

# Realizar predicciones
y_pred_final = lr_final.predict(X_optimal)

# Calcular métricas
mae_final = mean_absolute_error(y, y_pred_final)
mse_final = mean_squared_error(y, y_pred_final)
rmse_final = np.sqrt(mse_final)
r2_final = r2_score(y, y_pred_final)

print(f"\nMÉTRICAS DEL MODELO ÓPTIMO")
print("-"*80)
print(f"MAE  (Mean Absolute Error)     : ${mae_final:,.2f}")
print(f"MSE  (Mean Squared Error)      : ${mse_final:,.2f}")
print(f"RMSE (Root Mean Squared Error) : ${rmse_final:,.2f}")
print(f"R²   (Coeficiente Determinación): {r2_final:.4f} ({r2_final*100:.2f}%)")
print("-"*80)

# Interpretar R²
if r2_final > 0.9:
    interpretation = "Excelente"
elif r2_final > 0.7:
    interpretation = "Bueno"
elif r2_final > 0.5:
    interpretation = "Moderado"
else:
    interpretation = "Pobre"
    
print(f"\nInterpretación: El modelo explica el {r2_final*100:.2f}% de la variabilidad")
print(f"en las ventas totales. Ajuste: {interpretation}")

# Coeficientes del modelo
print(f"\nCOEFICIENTES DEL MODELO")
print("-"*80)
print(f"Intercepto (β₀): ${lr_final.intercept_:,.2f}")
print(f"\nCoeficientes de las variables:")
for i, (var, coef) in enumerate(zip(best_vars, lr_final.coef_), 1):
    print(f"   {i}. {var:30s}: β = {coef:12,.4f}")
print("="*80)

### 2.9 Visualización de Predicciones vs Valores Reales

In [ ]:
# Visualización de predicciones vs valores reales
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Gráfico 1: Scatter plot de predicciones vs valores reales
ax1 = axes[0, 0]
ax1.scatter(y, y_pred_final, alpha=0.5, s=20, color='steelblue')
ax1.plot([y.min(), y.max()], [y.min(), y.max()], 'r--', lw=2, label='Línea perfecta')
ax1.set_xlabel('Valores Reales', fontsize=12)
ax1.set_ylabel('Predicciones', fontsize=12)
ax1.set_title('Predicciones vs Valores Reales', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Gráfico 2: Distribución de residuos
residuals = y - y_pred_final
ax2 = axes[0, 1]
ax2.hist(residuals, bins=50, color='lightcoral', edgecolor='black', alpha=0.7)
ax2.axvline(x=0, color='red', linestyle='--', linewidth=2)
ax2.set_xlabel('Residuos', fontsize=12)
ax2.set_ylabel('Frecuencia', fontsize=12)
ax2.set_title('Distribución de Residuos', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

# Gráfico 3: Residuos vs predicciones
ax3 = axes[1, 0]
ax3.scatter(y_pred_final, residuals, alpha=0.5, s=20, color='purple')
ax3.axhline(y=0, color='red', linestyle='--', linewidth=2)
ax3.set_xlabel('Predicciones', fontsize=12)
ax3.set_ylabel('Residuos', fontsize=12)
ax3.set_title('Residuos vs Predicciones', fontsize=14, fontweight='bold')
ax3.grid(True, alpha=0.3)

# Gráfico 4: Q-Q plot (para normalidad de residuos)
from scipy import stats
ax4 = axes[1, 1]
stats.probplot(residuals, dist="norm", plot=ax4)
ax4.set_title('Q-Q Plot de Residuos', fontsize=14, fontweight='bold')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Estadísticas de residuos
print(f"\nANÁLISIS DE RESIDUOS")
print("-"*80)
print(f"Media de residuos        : ${np.mean(residuals):,.2f}")
print(f"Mediana de residuos      : ${np.median(residuals):,.2f}")
print(f"Desviación estándar      : ${np.std(residuals):,.2f}")
print(f"Residuo mínimo           : ${np.min(residuals):,.2f}")
print(f"Residuo máximo           : ${np.max(residuals):,.2f}")
print("-"*80)

---

## 3. Conclusiones

### 3.1 Hallazgos Principales

Este análisis ha demostrado exitosamente la aplicación del algoritmo Step Forward Feature Selection para la construcción de un modelo de regresión lineal múltiple predictivo de ventas retail.

In [ ]:
print("="*80)
print("RESUMEN DE CONCLUSIONES")
print("="*80)

print("\n1. DESEMPEÑO DEL ALGORITMO SFFS:")
print("-"*80)
print(f"   - Se evaluaron {len(potential_features)} variables candidatas")
print(f"   - El algoritmo identificó un conjunto óptimo de {best_n_vars} variables")
print(f"   - Tiempo de ejecución: {elapsed_time:.2f} segundos")
print(f"   - Mejora del MAE: {mae_improvement[best_idx-1]:.2f}%")

print("\n2. CALIDAD DEL MODELO FINAL:")
print("-"*80)
print(f"   - R² Score: {r2_final:.4f} ({interpretation})")
print(f"   - MAE: ${mae_final:,.2f}")
print(f"   - RMSE: ${rmse_final:,.2f}")
print(f"   - El modelo explica el {r2_final*100:.2f}% de la variabilidad en ventas")

print("\n3. VARIABLES MÁS IMPORTANTES:")
print("-"*80)
print("   Las primeras 3 variables seleccionadas por SFFS fueron:")
for i, var in enumerate(selected_vars[:min(3, len(selected_vars))], 1):
    print(f"   {i}. {var}")

print("\n4. APLICABILIDAD PRÁCTICA:")
print("-"*80)
print("   - El modelo puede utilizarse para predecir ventas futuras")
print("   - Las variables identificadas son clave para estrategias de negocio")
print("   - El SFFS permitió reducir dimensionalidad manteniendo precisión")

print("\n5. LIMITACIONES Y MEJORAS FUTURAS:")
print("-"*80)
print("   - Considerar validación cruzada para evaluar generalización")
print("   - Explorar transformaciones no lineales de variables")
print("   - Analizar posibles interacciones entre variables")
print("   - Evaluar modelos más complejos (Ridge, Lasso, etc.)")

print("="*80)

### 3.2 Interpretación de Resultados

El algoritmo **Step Forward Feature Selection** ha demostrado ser una herramienta efectiva para:

1. **Selección Automática**: Identificó sistemáticamente las variables más relevantes sin necesidad de conocimiento previo del dominio
2. **Optimización del Modelo**: Permitió encontrar el equilibrio entre complejidad y precisión
3. **Interpretabilidad**: Generó un ranking de importancia de variables, facilitando la comprensión del modelo
4. **Eficiencia Computacional**: Redujo la dimensionalidad del problema de manera estructurada

### 3.3 Recomendaciones

Con base en los resultados obtenidos, se recomienda:

- **Para el negocio**: Enfocar estrategias en las variables identificadas como más influyentes
- **Para análisis futuros**: Implementar validación cruzada para confirmar la robustez del modelo
- **Para mejoras del modelo**: Explorar técnicas de regularización (Ridge, Lasso) y modelos ensemble

### 3.4 Conclusión Final

Este estudio ha cumplido exitosamente con los objetivos propuestos, implementando un pipeline completo de análisis predictivo desde la exploración de datos hasta la evaluación del modelo final. El algoritmo SFFS ha probado ser una metodología sistemática y efectiva para la selección de características en regresión lineal múltiple, proporcionando resultados interpretables y accionables para la toma de decisiones en el contexto de ventas retail.

---

**Fin del Informe**